## Actividad 3: Scrapear Frases v2.0

En esta actividad 3 del módulo 3 scrapearemos un ecommerce extranjero.

Es importante que nos guiemos de la documentación: https://docs.scrapy.org/en/latest/ para saber qué está ocurriendo en el código y aprender nuevas formas de ejecutar el scraping

Con el código que vemos debajo podemos asegurarnos de instalar scrapy si es que no lo tenemos. Luego procedemos a importar el crawler que utilizaremos.

In [0]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
import re
import logging

Diseñamos la estructura de nuestro Item.

In [0]:
class EcommerceItem(scrapy.Item):
  title = scrapy.Field()
  cat = scrapy.Field()
  price = scrapy.Field()
  descr = scrapy.Field()

Diseñamos las funciones de nuestro Pipeline.
* ```process_item``` es la función que se llama para procesar el objeto.
* ```remove_tags``` es una función personalizada

In [0]:
# se va a trabajar con este pipeline
# 
class EcommercePipeline(object):
    def process_item(self, item, spider):
        return item
    
    def remove_tags(self, text):
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

Una vez tenemos definida nuestra Pipeline, procedemos a definir nuestro propio Spider. Atención a las ```custom_settings```.

In [0]:
# le ponemos un nombre al spider
class EcommerceSpider(scrapy.Spider):
    name = 'EcommerceSpider'
    allowed_domains = ['HarveyNorman.com.au']  # nuestro spider dice q se quede en esta web
    start_urls = ['https://www.HarveyNorman.com.au/']

    # log level para no llenarnos de mensajes
    #  hemos definido un item hasta aqui
    custom_settings = {
        'LOG_LEVEL': logging.WARNING, # Con esto reducimos el número de mensajes que recibimos
        'ITEM_PIPELINES': {'__main__.EcommercePipeline': 1}, # Ejecuta nuestra Pipeline
        'FEED_FORMAT':'csv',                                 # Almacenamos la información de nuestra Pipeline
        'FEED_URI': 'ecommerce.csv'
    }
# q es init? me permite añadir funcionalidad de constructor, se llena todos los valores de abajo
# son los expath a distintos elementos
    def __init__(self):
        #self.getAllCategoriesXpath = "//div[@id='wrapper']/div[1]/div[1]/div[1]/div[1]/div[contains(@class, 'col-md-3')]/ul/li/a/@href"
        #self.getAllSubCategoriesXpath = "//*[@id='content']/div[2]/div[1]/div/div[2]/div/div/div/div[2]/div/a/@href"
        self.getAllItemsXpath = "//*[@id='category-grid']/div/div/div[3]/a/@href" # es el selector q me traes todas las etiquetas de los productos de la portada, me pide el hipervinculo del producto
        #self.getAllItemsXpath = "//*[@id='category-grid']/div/div/div[@class='info']/a/@href" # por si cambia el nombre d euna clase
        self.titleXpath  = "//*[@id='overview']/div[1]/h1/span[1]/text()"
        self.categoryXpath = "//*[@id='breadcrumbs']/li/a/text()"
        self.priceXpath = "//div[contains(@class, 'product-view-sales')]//span[@class='price']/text()"
        self.descriptionXpath = "//*[@id='tab-content-product-description']/div/div[contains(@class,'description')][1]/p//text()"

 # porque hay 2 funciones de parse
 # va scrapear mi portada y va acalcular el hiperviculo
 # el segundo    
    def parse(self, response):
        for href in response.xpath(self.getAllItemsXpath): # q estrae? getxpath habia link en cada uno
            url = response.urljoin(href.extract()) # urljoin nos permite trabajar con la direccion completa
            yield scrapy.Request(url,callback=self.parse_main_item, dont_filter=True) # 
            # entrada x entrada, url=seria la web especifica de un producto
            # callback=yo hago este request sobre este url
            # donfilter = para q me salga todos los resultados vacios o no vacios

# primero inicializar un item
# y luego extraemos con nuestras expath
# es return ya q no esta iterando, se esta extrayendo de una pagina
    def parse_main_item(self,response):
        item = EcommerceItem()

        title = response.xpath(self.titleXpath).extract()
        category = response.xpath(self.categoryXpath).extract()
        price = response.xpath(self.priceXpath).extract()
        description = response.xpath(self.descriptionXpath).extract()

        item['title'] = title
        item['cat'] = category
        item['price'] = price
        item['descr'] = description
        return item

In [0]:
process = CrawlerProcess()

process.crawl(EcommerceSpider)
process.start()

2019-11-18 20:31:57 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2019-11-18 20:31:57 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.8 (default, Oct  7 2019, 12:59:55) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-11-18 20:31:57 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'ecommerce.csv', 'LOG_LEVEL': 30}
